In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet169, DenseNet121, DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import keras.backend as K
from sklearn.metrics import roc_auc_score, cohen_kappa_score
import cv2
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
import pydicom
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm
import csv
import math
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers 
from keras.layers.normalization import BatchNormalization as bn
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback

import keras.backend as K
import tensorflow as tf

from glob import glob
import matplotlib.pyplot as plt
from random import shuffle
%matplotlib inline

In [ ]:
def batch_generator(X,y, batch_size=8):
    '''
    Return a random image from X, y
    '''
    
    while True:
        # choose batch_size random images / labels from the data
        #idx = np.random.randint(0, X.shape[0], batch_size)
        i=0
        for idx in np.arange(8,X.shape[0], batch_size):
        
            im = np.array(X[i:idx])
            label = np.array(y[i:idx])
            




            yield im,label
            i = idx


In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
     self.val_f1s = []
     self.val_recalls = []
     self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
     val_predict = (np.asarray(self.model.predict(X_test_feats))).round()
     val_targ = Y_test
#      print(val_targ.shape)   
#      print(val_targ)
#      print(val_predict.shape)   
#      print(val_predict)   
     targ = []
     pred = []
     for x in val_targ:
        targ.append(np.argmax(x))
     for x in val_predict:
        pred.append(np.argmax(x))
#      print(targ)
#      print(pred)
        
     _val_f1 = f1_score(targ, pred)
     
     _val_recall = recall_score(targ, pred)
     _val_precision = precision_score(targ, pred)
     self.val_f1s.append(_val_f1)
     self.val_recalls.append(_val_recall)
     self.val_precisions.append(_val_precision)
     print("— val_f1: {0} — val_precision: {1} — val_recall {2} ".format(_val_f1, _val_precision, _val_recall))
     return

metrics = Metrics()

In [ ]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,validation_data):
#         self.x = training_data[0]
#         self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
#         y_pred = self.model.predict(self.x)
#         roc = roc_auc_score(self.y, y_pred)
        if(epoch%1==0):
            print("Calc Roc")
            all_rocs = []
            y_pred_val = self.model.predict(pad_X_test,verbose=1)
            for d in range(15):
                try:
                    roc_val = roc_auc_score(pad_Y_test[:,d], y_pred_val[:,d])
                    all_rocs.append(roc_val)
                except:
                    pass
            all_rocs = np.array(all_rocs)
            mean_roc = np.mean(all_rocs)
            
            f1s = {}
            for d in range(15):
                try:
                    score = f1_score(pad_Y_test[:,d], np.around(y_pred_val[:,d]))
                    f1s[global_cls[d]] = score
                except:
                    print(global_cls[d])
            avg_f1 = np.average(list(f1s.values()))
            
            print("ROC VAL {0}".format(mean_roc))
            print("AVG F1 {0}".format(avg_f1))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
def u_net(input_shape, dropout_rate, l2_lambda):
 
    # Encoder
    input = Input(shape = input_shape, name = "input")
    conv1_1 = Conv2D(32, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv1_1")(input)
    conv1_1 = bn(name = "conv1_1_bn")(conv1_1)
    conv1_2 = Conv2D(32, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv1_2")(conv1_1)
    conv1_2 = bn(name = "conv1_2_bn")(conv1_2)
    pool1 = MaxPooling2D(name = "pool1")(conv1_2)
    drop1 = Dropout(dropout_rate)(pool1)

    conv2_1 = Conv2D(64, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv2_1")(pool1)
    conv2_1 = bn(name = "conv2_1_bn")(conv2_1)
    conv2_2 = Conv2D(64, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv2_2")(conv2_1)
    conv2_2 = bn(name = "conv2_2_bn")(conv2_2)
    pool2 = MaxPooling2D(name = "pool2")(conv2_2)
    drop2 = Dropout(dropout_rate)(pool2)

    conv3_1 = Conv2D(128, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv3_1")(pool2)
    conv3_1 = bn(name = "conv3_1_bn")(conv3_1)
    conv3_2 = Conv2D(128, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv3_2")(conv3_1)
    conv3_2 = bn(name = "conv3_2_bn")(conv3_2)
    pool3 = MaxPooling2D(name = "pool3")(conv3_2)
    drop3 = Dropout(dropout_rate)(pool3)  

    conv4_1 = Conv2D(256, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv4_1")(pool3)
    conv4_1 = bn(name = "conv4_1_bn")(conv4_1)
    conv4_2 = Conv2D(256, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv4_2")(conv4_1)
    conv4_2 = bn(name = "conv4_2_bn")(conv4_2)
    pool4 = MaxPooling2D(name = "pool4")(conv4_2)
    drop4 = Dropout(dropout_rate)(pool4)  

    conv5_1 = Conv2D(512, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv5_1")(pool4)
    conv5_1 = bn(name = "conv5_1_bn")(conv5_1)
    conv5_2 = Conv2D(512, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv5_2")(conv5_1)
    conv5_2 = bn(name = "conv5_2_bn")(conv5_2)

    # Decoder
    upconv6 = Conv2DTranspose(256,(2, 2), strides=(2, 2), padding='same')(conv5_2)
    upconv6 = Dropout(dropout_rate)(upconv6)
    concat6 = concatenate([conv4_2, upconv6], name = "concat6")
    conv6_1 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_1")(concat6)
    conv6_1 = bn(name = "conv6_1_bn")(conv6_1)
    conv6_2 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_2")(conv6_1)
    conv6_2 = bn(name = "conv6_2_bn")(conv6_2)

    upconv7 = Conv2DTranspose(128,(2, 2), strides=(2, 2), padding='same')(conv6_2)
    upconv7 = Dropout(dropout_rate)(upconv7)
    concat7 = concatenate([conv3_2, upconv7], name = "concat7")
    conv7_1 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_1")(concat7)
    conv7_1 = bn(name = "conv7_1_bn")(conv7_1)
    conv7_2 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_2")(conv7_1)
    conv7_2 = bn(name = "conv7_2_bn")(conv7_2)

    upconv8 = Conv2DTranspose(64,(2, 2), strides=(2, 2), padding='same')(conv7_2)
    upconv8 = Dropout(dropout_rate)(upconv8)
    concat8 = concatenate([conv2_2, upconv8], name = "concat8")
    conv8_1 = Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_1")(concat8)
    conv8_1 = bn(name = "conv8_1_bn")(conv8_1)
    conv8_2 = Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_2")(conv8_1)
    conv8_2 = bn(name = "conv8_2_bn")(conv8_2)

    upconv9 = Conv2DTranspose(32,(2, 2), strides=(2, 2), padding='same')(conv8_2)
    upconv9 = Dropout(dropout_rate)(upconv9)
    concat9 = concatenate([conv1_2, upconv9], name = "concat9")
    conv9_1 = Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_1")(concat9)
    conv9_1 = bn(name = "conv9_1_bn")(conv9_1)
    conv9_2 = Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_2")(conv9_1)
    conv9_2 = bn(name = "conv9_2_bn")(conv9_2)
    dropout = Dropout(dropout_rate)(conv9_2)
   
    up6 = UpSampling2D(size=(8,8))(concat6)
    up7 = UpSampling2D(size=(4,4))(concat7)
    up8 = UpSampling2D(size=(2,2))(concat8)
    concate1 = concatenate([up6,up7,up8,dropout])

    conv10 = Conv2D(1, (1, 1), padding = "same", activation = 'sigmoid', name = "conv10")(concate1)


    model = Model(input, conv10)

    return model

In [ ]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [ ]:
X=np.load('../Numpy_Data/X_train_255.npy', mmap_mode='r+')
y=np.load('../Numpy_Data/Y_train_255.npy', mmap_mode='r+')

y_new=[]
for a in y1:
    y_new.append(cv2.cvtColor(a, cv2.COLOR_RGB2GRAY))

y=np.array(y_new)

y=y/255.0

y=np.reshape(y, (10675, 256,256,1))

In [ ]:
print (X.shape)
print(y.shape)

x=[]
Y=[]
for i in range(0, len(X)):
    x.append(cv2.resize(X[i], (128,128)))
    Y.append(cv2.resize(y[i], (128,128)))

x=np.array(x)
Y=np.array(Y)

In [ ]:
X_train=X[:8500]
X_valid=X[8500:]
Y_train=y[:8500]
Y_valid=y[8500:]

In [ ]:
del X
del y
del x
del Y

In [ ]:
def dice(true, preds):  
    scores  = []
    for i in range(len(true)):
        im1 = true[i]
        im2 = np.around(preds[i])
        intersection = np.logical_and(im1, im2)
        score = 2. * intersection.sum() / (im1.sum() + im2.sum())
        scores.append(score)
    return np.array(scores).mean()#, scores

def dice_score(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f)+K.epsilon())

def dice_loss(y_true, y_pred):
    return 1 - dice_score(y_true, y_pred)

In [ ]:
class dice_call(Callback):
    def __init__(self,validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs={}):

        if(epoch%1==0):
            print("Calc DICE")
            y_pred_val = np.around(self.model.predict(self.x_val,verbose=1))
            try:
                dice_val = dice(self.y_val, y_pred_val)
            except:
                pass
            print("DICE VAL {0}".format(dice_val))

        return

In [ ]:
smooth = 1.0
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f1 = K.flatten(K.round(y_pred))
    intersection = K.sum(y_true_f * y_pred_f1)
    return  (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f1) + smooth)

In [ ]:
input_shape = [256, 256, 3]
dropout_rate = 0.3
l2_lambda = 0.0002

#model = unet(None, input_shape)
model = u_net(input_shape, dropout_rate, l2_lambda)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [dice_coef])

In [ ]:
model.summary()

In [ ]:
dice_call1 = dice_call(validation_data=(X_valid, Y_valid))

In [ ]:
checkpoint = ModelCheckpoint('./Unet_updated_npy.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [ ]:
model.fit_generator(batch_generator(X_train, Y_train), validation_data=(X_valid, Y_valid), epochs=100, steps_per_epoch=len(X_train)/32, callbacks=callbacks, verbose=1)

In [ ]:
model.load_weights('./Unet_updated_npy.hdf5')

In [ ]:
X_test=np.load('../Numpy_Data/X_test.npy', mmap_mode='r+')

In [ ]:
X_test=X_test/255.0

In [ ]:
y_pred=model.predict(X_test, verbose=1, batch_size=8)

In [ ]:
y_pred.shape

In [ ]:
image=y_pred[1]
image=np.reshape(image, (256,256))
plt.imshow(image, cmap='gray')

In [ ]:
test_csv=pd.read_csv('../SIIM_test.csv')

In [ ]:
predictions=y_pred

In [ ]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

In [ ]:
def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)

In [ ]:
test_csv=test_csv.drop(columns='Unnamed: 0')

In [ ]:
test_csv.columns

In [ ]:
mask_new=rle2mask(image_encoded[test_csv['0'][1]],256,256)

In [ ]:
plt.imshow(np.reshape(y_pred[0], (256,256)))

In [ ]:
plt.imshow(mask_new, 'gray')

In [ ]:
image_encoded={}
for index, image in enumerate(y_pred):
    if np.max(image)==0:
        image_encoded[test_csv['0'][index]]=-1
        
    else:
        image=np.reshape(image, (256,256))
        image_encoded[test_csv['0'][index]]=mask2rle(image, 256, 256)
    print (index)

In [ ]:
len(image_encoded)

In [ ]:
df=pd.DataFrame.from_dict(image_encoded, orient='index')

In [ ]:
df_new = pd.DataFrame()

In [ ]:
list_of_images=[]
for i in test_csv['0']:
    a=i.split('/')[-1]
    a=a.split('.png')
    list_of_images.append(a[0])

In [ ]:
print (image_encoded)

In [ ]:
df_new['ImageId']=list_of_images

In [ ]:
df_new['EncodedPixels']=image_encoded.values()

In [ ]:
df_new.to_csv('./submission.csv', index=False)

In [ ]:
y_pred_1=y_pred_1.astype('uint8')

In [ ]:
i=150
plt.imshow(np.around(y_pred[i]), cmap='gray')

In [ ]:
plt.imshow(Y_valid[i], cmap='gray')

In [ ]:
np.around(y_pred[i])

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16,16))
columns = 4
rows = 5
j=0
for i in range(1, columns*rows +1):
    img = np.random.randint(1,2000)
    fig.add_subplot(rows, columns, i)
    plt.imshow(np.around(y[img]), cmap='gray')
    j=j+1
plt.show()

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16,16))
columns = 4
rows = 5
j=0
for i in range(1, columns*rows +1):
    img = np.random.randint(1,2000)
    fig.add_subplot(rows, columns, i)
    plt.imshow(Y_train[j], cmap='gray')
    j=j+1
plt.show()